<a href="https://colab.research.google.com/github/Piripack/business-bot/blob/main/run_business_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script handles user interactions for a business bot. It connects to a SQLite database to verify user identity and respond to user queries about personal data, appointments, products, or FAQs.

In [ ]:
import sqlite3
import re


Imports: Importing the sqlite3 module for database operations and the re module for regular expressions.

In [ ]:
def get_db_connection():
    conn = sqlite3.connect('business_bot.db')
    return conn


Database Connection: Defines a function that connects to the business_bot.db SQLite database and returns the connection object.

In [ ]:
def execute_query(query, params=()):
    conn = get_db_connection()
    c = conn.cursor()
    c.execute(query, params)
    results = c.fetchall()
    conn.close()
    return results


Execute SQL Query: Defines a function that executes a given SQL query with optional parameters, fetches all results, closes the database connection, and returns the results.

In [ ]:
def question_to_sql(question, email=None):
    question = question.lower()

    if email:
        if "id" in question:
            return "SELECT id FROM users WHERE email = ?", (email,)
        if "password" in question:
            return "SELECT password FROM users WHERE email = ?", (email,)
        if "dob" in question or "date of birth" in question:
            return "SELECT dob FROM users WHERE email = ?", (email,)
        if "favorite color" in question or "fav color" in question:
            return "SELECT fav_color FROM users WHERE email = ?", (email,)
        if "name" in question:
            return "SELECT name FROM users WHERE email = ?", (email,)
        if "phone" in question:
            return "SELECT phone FROM users WHERE email = ?", (email,)
        if "address" in question:
            return "SELECT address FROM users WHERE email = ?", (email,)
        if "registration date" in question:
            return "SELECT registration_date FROM users WHERE email = ?", (email,)
        if "last login" in question:
            return "SELECT last_login FROM users WHERE email = ?", (email,)

    if "appointment" in question:
        if "schedule" in question or "book" in question:
            return "INSERT INTO appointments (user_id, date, time, description) VALUES (?, ?, ?, ?)"
        if "view" in question or "list" in question:
            return "SELECT * FROM appointments"

    if "product" in question:
        if "price" in question:
            product_match = re.search(r'product\\s*=\\s*(\\w+)', question)
            if product_match:
                product_name = product_match.group(1)
                return "SELECT price FROM products WHERE name = ?", (product_name,)

    if "faq" in question or "question" in question:
        return "SELECT question, answer FROM faqs"

    return None, None


Translate Question to SQL Query: Defines a function that translates user questions into corresponding SQL queries based on the question content and email provided.

In [ ]:
def verify_email(email):
    query = "SELECT email FROM users WHERE email = ?"
    params = (email,)
    results = execute_query(query, params)

    if results:
        return True
    else:
        return False

Verify Email: Defines a function that checks if an email exists in the users table of the database.

In [ ]:
def verify_user():
    while True:
        email = input("Bot: Please enter your email: ").strip()

        if verify_email(email):
            print("Bot: Email found in our system.")
            while True:
                password = input("Bot: Please enter your password: ").strip()
                query = "SELECT * FROM users WHERE email = ? AND password = ?"
                params = (email, password)
                results = execute_query(query, params)

                if results:
                    print("Bot: You are successfully logged in!")
                    return True, email
                else:
                    print("Bot: Incorrect password. Please try again.")
        else:
            print("Bot: This email is not registered in our system.")
            user_input = input("Bot: Would you like to try a different email? (yes/no): ").strip().lower()
            if user_input in ["no", "bye", "exit", "quit", "goodbye", "thanks"]:
                print("Bot: Goodbye! Have a great day!")
                return False, None


Verify User: Defines a function that prompts the user to enter their email and password, verifies their credentials, and allows retrying on failure. If the email is not found, it offers to retry with a different email or exit.

In [ ]:
def handle_greeting():
    print("Bot: Hello! How can I help you today?")

    while True:
        user_input = input("User: ").strip().lower()

        if user_input in ["hi", "hello", "hey"]:
            print("Bot: How can I assist you today? You can ask about appointments, products, or FAQs.")
            user_input = input("User: ").strip().lower()

            print("Bot: I can help with that. But first, I need to verify your identity.")
            verified, email = verify_user()
            if verified:
                query, params = question_to_sql(user_input, email)
                if query:
                    results = execute_query(query, params)
                    if results:
                        for row in results:
                            print(f"Bot: Your {user_input} is {row[0]}")
                    else:
                        print(f"Bot: No information found for {user_input}.")

                while True:
                    print("Bot: How can I assist you further? You can ask about your data, appointments, products, or FAQs.")
                    user_input = input("User: ").strip().lower()
                    query, params = question_to_sql(user_input, email)
                    if query:
                        results = execute_query(query, params)
                        if results:
                            for row in results:
                                print(f"Bot: Your {user_input} is {row[0]}")
                        else:
                            print(f"Bot: No information found for {user_input}.")
                    else:
                        print("Bot: I'm sorry, I don't understand your question.")

                    if user_input in ["no", "thank you", "bye", "exit", "quit", "goodbye", "thanks"]:
                        print("Bot: You're welcome! Have a great day!")
                        break


Handle Greeting and Responses: Defines a function that greets the user, processes initial input, verifies user identity, and handles further queries in a loop.

In [ ]:
def bot_interaction():
    while True:
        handle_greeting()

        user_input = input("User: ").strip().lower()
        if user_input in ["no", "thank you", "bye", "exit", "quit", "goodbye", "thanks"]:
            print("Bot: You're welcome! Have a great day!")
            break


Bot Interaction Loop: Defines a function that runs the bot interaction in a loop, continuing until the user indicates they are done.

In [ ]:
bot_interaction()


Start Bot Interaction: Initiates the bot interaction by calling the bot_interaction function.